# Evaluation of RAG model

One way to think about different types of RAG evaluators is as a tuple of what is being evaluated X what its being evaluated against:

1. Correctness: Response vs reference answer
- Goal: Measure "how similar/correct is the RAG chain answer, relative to a ground-truth answer"
- Mode: Requires a ground truth (reference) answer supplied through a dataset
- Evaluator: Use LLM-as-judge to assess answer correctness.

2. Relevance: Response vs input
- Goal: Measure "how well does the generated response address the initial user input"
- Mode: Does not require reference answer, because it will compare the answer to the input question
- Evaluator: Use LLM-as-judge to assess answer relevance, helpfulness, etc.

3. Groundedness: Response vs retrieved docs
- Goal: Measure "to what extent does the generated response agree with the retrieved context"
- Mode: Does not require reference answer, because it will compare the answer to the retrieved context
- Evaluator: Use LLM-as-judge to assess faithfulness, hallucinations, etc.

4. Retrieval relevance: Retrieved docs vs input
- Goal: Measure "how relevant are my retrieved results for this query"
- Mode: Does not require reference answer, because it will compare the question to the retrieved context
- Evaluator: Use LLM-as-judge to assess relevance

In [1]:
from langchain_google_genai import GoogleGenerativeAI
import google.generativeai as genai
import json
from pydantic import BaseModel
from typing_extensions import Annotated, TypedDict, Type
from langsmith import Client
import os
import pickle as pkl
import helper_functions as hf
from langchain_huggingface import HuggingFaceEmbeddings
import glob
import re
from tqdm import tqdm
import pandas as pd
import numpy as np

/home/antonioparragaleo/miniconda3/envs/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('api_google.txt') as f:
    
    api_key = json.load(f)

## Setting

In [3]:
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = api_key['key']

In [4]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0)

In [5]:
with open("generated_questions_final_gemini_2-5.pkl", "rb") as f:
    eval_datasets = pkl.load(f)

## Evaluation

In [6]:
# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a senior researcher in reprodutive field and bioinformatics. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the PhD Student answer. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the PhD student's answer meets all of the criteria.
A correctness value of False means that the PhD student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM

grader_llm = llm.with_structured_output(CorrectnessGrade, method="json_schema", strict=True)

In [7]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Grade prompt
relevance_instructions="""You are a senior researcher in reprodutive field and bioinformatics. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the PhD STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the PhD STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

relevance_llm = llm.with_structured_output(RelevanceGrade, method="json_schema", strict=True)

In [8]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are a senior researcher in reprodutive field and bioinformatics grading PhD students questions. 

You will be given a QUESTION and a set of FACTS provided by the PhD student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_llm = llm.with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

In [9]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = """You are a senior researcher in reprodutive field and bioinformatics grading PhD students questions. 

You will be given FACTS and a PhD STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the PhD STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the PhD STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM 
grounded_llm = llm.with_structured_output(GroundedGrade, method="json_schema", strict=True)


In [37]:
outputs = {}
for file in glob.glob("./output/*.json"):
    output = json.load(open(file, "r"))
    outputs[re.sub(r'./output/rag_','',file)] = output


In [39]:
try:
    with open("./evaluation/evaluation.pkl","rb") as f:
        list_dataframes = pkl.load(f)
except:
    print('File not found')
    list_dataframes = []

In [ ]:
if os.path.exists("./evaluation/evaluation_log.txt"):
    with open("./evaluation/evaluation_log.txt", "r") as f:
        lines = [line.strip() for line in f]
else:
    lines = []

for settings, evaluation in tqdm(outputs.items(), total=len(outputs)):

    if settings in lines:
        print(f"Settings: {settings} already analyzed")
        continue
    
    print(f"Analyzing {settings}")
    aux_correct = []
    aux_relevant = []
    aux_retrival = []
    aux_ground = []

    for mydic in evaluation:
        aux_correct.append(hf.correctness(inputs=mydic, grader_llm = grader_llm, correctness_instructions= correctness_instructions))
        aux_relevant.append(hf.relevance(inputs=mydic, relevance_llm  = relevance_llm, relevance_instructions = relevance_instructions))
        aux_retrival.append(hf.retrieval_relevance(inputs=mydic, retrieval_llm= retrieval_llm, retrieval_relevance_instructions = retrieval_relevance_instructions))
        aux_ground.append(hf.groundedness(inputs=mydic, grounded_llm=grounded_llm, grounded_instructions=grounded_instructions))

    # Create dataframe
    correctness_results = pd.Series(aux_correct, name="Correctness")
    relevance_results = pd.Series(aux_correct, name="Relevance")
    retrieval_results = pd.Series(aux_retrival, name="Retrieval")
    ground_results = pd.Series(aux_ground, name="Groundedness")
    list_dataframes.append(pd.concat([pd.DataFrame.from_dict(evaluation),correctness_results, relevance_results, retrieval_results, ground_results], axis = 1))

    with open("./evaluation/evaluation.pkl","wb") as f:
        pkl.dump(list_dataframes,f)

    with open("./evaluation/evaluation_log.txt","a") as f:
        f.write(f"{settings}\n")

  0%|          | 0/126 [00:00<?, ?it/s]

Settings: chunk_1000_reader-model_gemini-2.0-flash_emnedding_model_e5-small-v2recursive_True_k_8.json already analyzed
Settings: chunk_1500_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_False_k_9.json already analyzed
Analyzing chunk_1500_reader-model_gemini-2.0-flash_emnedding_model_GIST-small-Embedding-v0recursive_False_k_8.json


  2%|▏         | 3/126 [03:29<2:23:08, 69.82s/it]

Analyzing chunk_1000_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_False_k_4.json


  3%|▎         | 4/126 [06:49<3:47:38, 111.95s/it]

...

Analyzing chunk_1500_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_False_k_7.json


 10%|█         | 13/126 [37:28<6:18:27, 200.95s/it]

Analyzing chunk_2000_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_True_k_6.json


 11%|█         | 14/126 [41:00<6:21:29, 204.37s/it]

Analyzing chunk_1000_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_True_k_8.json


In [14]:
toplots = []
for subset in list_dataframes:
    toplot = subset.loc[:,['test_settings','eval_score_gemini-2.0-flash','Relevance','Correctness','Groundedness','Retrieval']]
    values = [toplot.test_settings[0].split('_')[i] for i in [1,3,6,7,9]]
    toplot2 = {"Chunk_size": np.repeat(values[0], toplot.shape[0]),
    "LLM_model":np.repeat(values[1], toplot.shape[0]),
    "Embedding_Model":np.repeat(re.sub(r'recursive','',values[2]),toplot.shape[0]),
    "Recursive":np.repeat(values[3], toplot.shape[0]),
    "NumberofDocuments":np.repeat(values[4], toplot.shape[0])}
    toplots.append(pd.concat([pd.DataFrame(toplot2), toplot.loc[:,['eval_score_gemini-2.0-flash','Relevance','Correctness','Groundedness','Retrieval']]], axis=1))
toplot_final = pd.concat(toplots)

In [15]:
toplot_final["eval_score_gemini-2.0-flash"] = toplot_final["eval_score_gemini-2.0-flash"].apply(lambda x: int(x) if isinstance(x, str) else 0)
toplot_final["eval_score_gemini-2.0-flash"] = toplot_final["eval_score_gemini-2.0-flash"]/5

In [18]:
toplot_final.groupby(["Chunk_size","Embedding_Model", "Recursive", "NumberofDocuments"])["Relevance"].mean()

Chunk_size  Embedding_Model           Recursive  NumberofDocuments
1000        e5-small-v2               True       8                    0.984127
1500        snowflake-arctic-embed-s  False      9                    0.714286
Name: Relevance, dtype: float64

In [19]:
toplot_final.query('Chunk_size == "1000" and Embedding_Model == "e5-small-v2" and Recursive == "True"').Relevance.mean()

np.float64(0.9841269841269841)